# Jupyter Notebook für Test Cases

In [ ]:
%run ../../Setup.ipynb

import unittest
import subprocess
from testfunctions_notebook import cuboid_volume_notebook, division_notebook, division2_notebook

In [ ]:
class TestCaseNotebook(unittest.TestCase):

    def test_volume_list(self):
        vals = [[2,8], [1,1], [0,0], [5.5, 166.375]]
        for val in vals:
            self.assertAlmostEqual(cuboid_volume_notebook(val[0]), val[1])

    def test_volume(self):
        self.assertAlmostEqual(cuboid_volume_notebook(2),8)
        self.assertAlmostEqual(cuboid_volume_notebook(1),1)
        self.assertAlmostEqual(cuboid_volume_notebook(0),0)
        self.assertAlmostEqual(cuboid_volume_notebook(5.5),166.375)

    def test_division(self):
        self.assertAlmostEqual(division_notebook(4,2), 2)
    
    def test_division2(self):
        self.assertAlmostEqual(division2_notebook(4,2), 2)
    
    def test_division3(self):
        self.assertAlmostEqual(division_notebook(100,50), 2)

    def test_division_zero(self):
        self.assertRaises(ZeroDivisionError, lambda: division_notebook(4,0))
    
    def test_division2_zero(self):
        self.assertRaises(ZeroDivisionError, lambda: division2_notebook(4,0))

In [ ]:
def suite():
    """Funktion zum Erstellen einer Test Suite für die Ausführung der Unittests."""
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(TestCaseNotebook, 'test'))
    return suite

In [ ]:
def execute_unittests():
    """Funktion zum Ausführen der Unittests."""
    unittest.main(argv=[''], verbosity=2, exit=False, defaultTest='suite')

In [ ]:
def execute_coverage():
    # TBD
    pass
    # os.system("coverage run -m testcases.py")
    # subprocess.run(["coverage", "run", "-m", "testcases.py"])
    # result = subprocess.run(["coverage", "report", "-m"], stdout=subprocess.PIPE)
    # output = result.stdout
    # output = output.decode("utf-8")
    # print(output)
    # subprocess.run(["coverage", "html", "-d", "html"])
    # cov = coverage.Coverage()
    # cov.start()
    # unittest.main(argv=[''], verbosity=2, exit=False, defaultTest='suite')
    # cov.stop()
    # cov.save()
    # cov.html_report()